# Assignment 4

## Implement regression tree algorithm from scratch

- Recursive binary splitting: We first take the predicator $X_j$ and the cut point $s$ such that we split the space in two regions ${X|X_j < s}$ and ${X|X_j > s}$ leads to the **greatest possible reduction in RSS**.
- For any $j$ and $s$ we define the pair of half-planes
- So if our data set contains many features $(X_n)_{n \in \mathbb N}$, I think we do some cross validation on all splits ${X|X_j < s}$ and ${X|X_j > s}$ for all $0 \le j \le n$ and then take make a node on the one that split it with the greatest reduction in RSS.

The goal is to find the boxes $R1,\dots,R_J$ that minimze the RSS.

-> **Question**: how do we find the right amount of boxes $J$? 
- We have to put a criteria of minimum data in one box. Usually its around 20 from what I recall.